In [1]:
# STK library imports
from agi.stk12.stkdesktop import STKDesktop
from agi.stk12.stkobjects import *
from agi.stk12.stkutil import *
from agi.stk12.vgt import *
# if using astrogator uncomment the below
# from agi.stk12.stkobjects.astrogator
# if using aviator uncomment the below
# from agi.stk12.stkobjects.aviator

# Python helper library imports
import os

In [2]:
# Get reference to the current instance of STK
STK_PID = os.getenv('STK_PID')
stk = STKDesktop.AttachToApplication(pid=int(STK_PID))
root = stk.Root

In [3]:
scenario = root.CurrentScenario # 链接当前场景
scenario.SetTimePeriod('Today','+24hr') # 日期设置为今天
root.Rewind() # 复位

In [4]:
print(type(root))

<class 'agi.stk12.stkobjects.AgStkObjectRoot'>


In [5]:
from comtypes.gen import STKObjects

In [6]:
satNums = scenario.Children.GetElements(STKObjects.eSatellite)
num = satNums.Count
print(num)

2000


In [7]:
for i in range(num):
    sat = scenario.Children.Item(i)
    satPath = sat.Path
    print(satPath)

/Application/STK/Scenario/Starlink_2000/Chain/Chain1
/Application/STK/Scenario/Starlink_2000/Constellation/Starlink
/Application/STK/Scenario/Starlink_2000/Constellation/Station
/Application/STK/Scenario/Starlink_2000/Place/Harbin
/Application/STK/Scenario/Starlink_2000/Satellite/DARKSAT_44932
/Application/STK/Scenario/Starlink_2000/Satellite/STARLINK-1008_44714
/Application/STK/Scenario/Starlink_2000/Satellite/STARLINK-1010_44716
/Application/STK/Scenario/Starlink_2000/Satellite/STARLINK-1011_44717
/Application/STK/Scenario/Starlink_2000/Satellite/STARLINK-1012_44718
/Application/STK/Scenario/Starlink_2000/Satellite/STARLINK-1013_44719
/Application/STK/Scenario/Starlink_2000/Satellite/STARLINK-1014_44720
/Application/STK/Scenario/Starlink_2000/Satellite/STARLINK-1015_44721
/Application/STK/Scenario/Starlink_2000/Satellite/STARLINK-1017_44723
/Application/STK/Scenario/Starlink_2000/Satellite/STARLINK-1019_44724
/Application/STK/Scenario/Starlink_2000/Satellite/STARLINK-1020_44725
/Appl

In [8]:
satNums

In [10]:
for each in satNums: #遍历已有的卫星
        Instance_name = each.InstanceName
        #  new transmitter and receiver
        transmitter = each.Children.New(STKObjects.eTransmitter, "Transmitter_" + Instance_name)

In [16]:
place = scenario.Children.GetElements(STKObjects.ePlace)
place = scenario.Children.Item(3)
placePath = place.Path
print(place.Path)

/Application/STK/Scenario/Starlink_2000/Place/Harbin


In [18]:
place = root.GetObjectFromPath(placePath)
print(place.Children.Item(0).Path)

/Application/STK/Scenario/Starlink_2000/Place/Harbin/Receiver/Receiver1


In [19]:
groundReceiver = root.GetObjectFromPath('/Application/STK/Scenario/Starlink_2000/Place/Harbin/Receiver/Receiver1')

In [37]:
from tqdm import tqdm
def Set_Transmitter_Parameter(transmitter, frequency=26.5, EIRP=20, DataRate=14):
    #transmitter2 = transmitter.QueryInterface(STKObjects.IAgTransmitter)  # 建立发射机的映射，以便对其进行设置
    transmitter.SetModel('Simple Transmitter Model')
    txModel = transmitter.Model
    #txModel = txModel.QueryInterface(STKObjects.IAgTransmitterModelSimple)
    txModel.Frequency = frequency  # GHz range:10.7-12.7GHz
    #txModel.EIRP = EIRP  # dBW
    #txModel.DataRate = DataRate  # Mb/sec
for each_sat in tqdm(satNums):
    now_sat_name = each_sat.InstanceName
    now_sat_transmitter = each_sat.Children.GetElements(STKObjects.eTransmitter)[0]  # 找到该卫星的发射机
    Set_Transmitter_Parameter(now_sat_transmitter, EIRP=20)
    access = groundReceiver.GetAccessToObject(now_sat_transmitter)
    access.ComputeAccess()

2000it [52:45,  1.58s/it]
